In [107]:
import numpy as np
import pandas as pd

In [108]:
anime=pd.read_csv("anime-dataset-2023.csv")

In [109]:
anime.sample(4)

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
9311,25533,Many Mary,UNKNOWN,メニー・メアリー,4.64,UNKNOWN,"Music video for the song ""Many Mary"" by A Crow...",Music,1.0,"Apr 14, 2013",...,UNKNOWN,Unknown,2 min,G - All Ages,UNKNOWN,15112,0,270.0,480,https://cdn.myanimelist.net/images/anime/13/65...
3431,4013,Paul no Miracle Daisakusen,Paul in Fantasy Island,ポールのミラクル大作戦,6.36,"Action, Adventure",Paul is a young boy whose friend Nina is kidna...,TV,50.0,"Oct 3, 1976 to Sep 11, 1977",...,Tatsunoko Production,Original,25 min per ep,G - All Ages,7105.0,12416,1,340.0,1068,https://cdn.myanimelist.net/images/anime/3/564...
1529,1682,Baldr Force Exe Resolution,UNKNOWN,バルドフォース エグゼ レソリューション,6.4,"Action, Drama, Sci-Fi","Any network runs all over the world, and the d...",OVA,4.0,"Nov 10, 2006 to Apr 4, 2007",...,Satelight,Visual novel,29 min per ep,R+ - Mild Nudity,6886.0,4911,26,7212.0,16929,https://cdn.myanimelist.net/images/anime/3/762...
16873,42198,Atashin'chi Specials,UNKNOWN,あたしンち SP,UNKNOWN,"Comedy, Slice of Life",Special episodes of Atashin'chi.,Special,4.0,"Oct 11, 2002 to Oct 30, 2004",...,UNKNOWN,Manga,22 min per ep,G - All Ages,14653.0,18443,0,UNKNOWN,226,https://cdn.myanimelist.net/images/anime/1666/...


In [110]:
anime.describe()

,anime_id,Popularity,Favorites,Members
count,24905.000000,24905.000000,24905.000000,2.490500e+04
mean,29776.709014,12265.388356,432.595222,3.710496e+04
std,17976.076290,7187.428393,4353.181647,1.568252e+05
min,1.000000,0.000000,0.000000,0.000000e+00
25%,10507.000000,6040.000000,0.000000,2.090000e+02
50%,34628.000000,12265.000000,1.000000,1.056000e+03
75%,45240.000000,18491.000000,18.000000,9.326000e+03
max,55735.000000,24723.000000,217606.000000,3.744541e+06


In [111]:
anime.shape

(24905, 24)

In [112]:
# Select only numeric columns automatically
anime.corr(numeric_only=True)

,anime_id,Popularity,Favorites,Members
anime_id,1.000000,0.510459,-0.054634,-0.091948
Popularity,0.510459,1.000000,-0.163173,-0.363865
Favorites,-0.054634,-0.163173,1.000000,0.773120
Members,-0.091948,-0.363865,0.773120,1.000000


In [113]:
anime.isnull().sum()

anime_id        0
Name            0
English name    0
Other name      0
Score           0
Genres          0
Synopsis        0
Type            0
Episodes        0
Aired           0
Premiered       0
Status          0
Producers       0
Licensors       0
Studios         0
Source          0
Duration        0
Rating          0
Rank            0
Popularity      0
Favorites       0
Scored By       0
Members         0
Image URL       0
dtype: int64

In [114]:
anime['Members']=pd.to_numeric(anime['Members'],errors='coerce')
anime=anime.sort_values(by='Members',ascending=False).head(10000).copy()
anime=anime.reset_index(drop=True)

In [115]:
anime=anime[['anime_id','Name','English name','Genres','Synopsis','Type','Source','Studios','Image URL']]

In [116]:
anime.shape

(10000, 9)

In [117]:
anime['tags']=(anime['Name']+' '+anime['English name']+' '+anime['Genres']+' '+anime['Synopsis']+' '+anime['Type']+' '+anime['Source']+' '+anime['Studios']+' '+anime['Type'])
anime['tags'].apply(lambda x: x.lower())


0       shingeki no kyojin attack on titan action, awa...
1       death note death note supernatural, suspense b...
2       fullmetal alchemist: brotherhood fullmetal alc...
3       one punch man one punch man action, comedy the...
4       sword art online sword art online action, adve...
                              ...                        
9995    shining hearts: shiawase no pan specials shini...
9996    soreyuke! uchuu senkan yamamoto youko ii stars...
9997    gekijou tanpen macross frontier: toki no meiky...
9998    the idolm@ster million live! 4-shuunen kinen p...
9999    doukyuusei remake the animation unknown hentai...
Name: tags, Length: 10000, dtype: object

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [119]:
tfid=TfidfVectorizer(max_features=6000,stop_words='english')
vector=tfid.fit_transform(anime['tags']).toarray()

In [120]:
from sklearn.metrics.pairwise import cosine_similarity

In [121]:
similarity=cosine_similarity(vector)

In [126]:
similarity.shape

(10000, 10000)

In [144]:
def recommend_anime(anime_name,n=9):
    idx=anime[anime['Name']==anime_name].index[0]
    distances=similarity[idx]
    anime_list=sorted(list(enumerate(distances)),key=lambda x:x[1],reverse=True)[5:n+1]
    for i in anime_list:
        print(anime.iloc[i[0]]['Name'])

In [145]:
recommend_anime('Steins;Gate')

Steins;Gate 0
Steins;Gate 0: Kesshou Takei no Valentine - Bittersweet Intermedio
Gate: Jieitai Kanochi nite, Kaku Tatakaeri
Kichiku: Haha Shimai Choukyou Nikki
Fight League: Gear Gadget Generators


In [146]:
pickle.dump(anime,open('anime.pkl','wb'))
pickle.dump(similarity,open('anime_similarity.pkl','wb'))